## PLX - Phuture Liquid Staking Index - Methodology Template

## Configuring packages

In [1]:
import pandas as pd
import numpy as np
import requests
import decouple
import sys 
sys.path.append('../')
import datetime 
import time
from methodology import Methodology
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

## Index Variables

In [2]:
min_mcap =  0.5e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.2
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = "liquid-staking-governance-tokens"

# Calculating max slippage based on liquidity threshold
liq = 1.5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)
max_slippage

stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = Methodology(min_mcap,min_weight,max_weight,circ_supply_threshold,liveness_threshold,liquidity_consistency,max_slippage,[stablecoins,ctokens,atokens],coingecko_category)

## Data retrieval and filtering

### Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $150mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 30% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 4% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


#### Supported assets and blockchains

#### CoinGecko

##### Gathering market data

In [3]:
methodology_class.get_category_data()

,symbol,name,current_price,market_cap,market_cap_rank,fully_diluted_valuation,circulating_supply,total_supply,max_supply
id,,,,,,,,,
lido-dao,ldo,Lido DAO,2.370000,2.105527e+09,37.0,2.365619e+09,8.900535e+08,1.000000e+09,1.000000e+09
frax-share,fxs,Frax Share,8.590000,6.519259e+08,93.0,8.622444e+08,7.536720e+07,9.968150e+07,9.968150e+07
rocket-pool,rpl,Rocket Pool,30.790000,6.127544e+08,96.0,6.127544e+08,1.991675e+07,1.991675e+07,NaN
ankr,ankr,Ankr Network,0.028171,2.812949e+08,174.0,2.812949e+08,1.000000e+10,1.000000e+10,1.000000e+10
stride,strd,Stride,2.180000,1.924208e+08,222.0,1.924218e+08,8.830823e+07,8.830870e+07,1.000000e+08
bifrost,bfc,Bifrost,0.053423,7.388816e+07,424.0,1.263909e+08,1.384675e+09,2.368584e+09,2.368584e+09
veno-finance,vno,Veno Finance,0.162685,6.636967e+07,453.0,2.967988e+08,4.103687e+08,1.835130e+09,2.000000e+09
marinade,mnde,Marinade,0.232943,5.619691e+07,498.0,2.333945e+08,2.407807e+08,1.000000e+09,1.000000e+09


In [4]:
methodology_class.add_assets_to_category(['benqi','stader','stakewise','stake-dao','yearn-finance','yield-yak'])

,symbol,name,current_price,market_cap,market_cap_rank,fully_diluted_valuation,circulating_supply,total_supply,max_supply
id,,,,,,,,,
lido-dao,ldo,Lido DAO,2.370000,2.105527e+09,37.0,2.365619e+09,8.900535e+08,1.000000e+09,1.000000e+09
frax-share,fxs,Frax Share,8.590000,6.519259e+08,93.0,8.622444e+08,7.536720e+07,9.968150e+07,9.968150e+07
rocket-pool,rpl,Rocket Pool,30.790000,6.127544e+08,96.0,6.127544e+08,1.991675e+07,1.991675e+07,NaN
yearn-finance,yfi,yearn.finance,8594.180000,2.842358e+08,172.0,3.135896e+08,3.323384e+04,3.666600e+04,3.666600e+04
ankr,ankr,Ankr Network,0.028171,2.812949e+08,174.0,2.812949e+08,1.000000e+10,1.000000e+10,1.000000e+10
stride,strd,Stride,2.180000,1.924208e+08,222.0,1.924218e+08,8.830823e+07,8.830870e+07,1.000000e+08
bifrost,bfc,Bifrost,0.053423,7.388816e+07,424.0,1.263909e+08,1.384675e+09,2.368584e+09,2.368584e+09
veno-finance,vno,Veno Finance,0.162685,6.636967e+07,453.0,2.967988e+08,4.103687e+08,1.835130e+09,2.000000e+09
marinade,mnde,Marinade,0.232943,5.619691e+07,498.0,2.333945e+08,2.407807e+08,1.000000e+09,1.000000e+09


In [5]:
methodology_class.get_all_coin_data()
methodology_class.filter_and_merge_coin_data()
methodology_class.token_supply_check()
methodology_class.asset_maturity_check()
methodology_class.remove_manual_exclusions([])

,symbol,name,current_price,market_cap,market_cap_rank,fully_diluted_valuation,circulating_supply,total_supply,max_supply,platforms
id,,,,,,,,,,
lido-dao,ldo,Lido DAO,2.370000,2.105527e+09,37.0,2.365619e+09,8.900535e+08,1.000000e+09,1.000000e+09,{'ethereum': '0x5a98fcbea516cf06857215779fd812...
frax-share,fxs,Frax Share,8.590000,6.519259e+08,93.0,8.622444e+08,7.536720e+07,9.968150e+07,9.968150e+07,{'ethereum': '0x3432b6a60d23ca0dfca7761b7ab564...
rocket-pool,rpl,Rocket Pool,30.790000,6.127544e+08,96.0,6.127544e+08,1.991675e+07,1.991675e+07,NaN,{'ethereum': '0xd33526068d116ce69f19a9ee46f0bd...
yearn-finance,yfi,yearn.finance,8594.180000,2.842358e+08,172.0,3.135896e+08,3.323384e+04,3.666600e+04,3.666600e+04,{'ethereum': '0x0bc529c00c6401aef6d220be8c6ea1...
ankr,ankr,Ankr Network,0.028171,2.812949e+08,174.0,2.812949e+08,1.000000e+10,1.000000e+10,1.000000e+10,{'ethereum': '0x8290333cef9e6d528dd5618fb97a76...
bifrost,bfc,Bifrost,0.053423,7.388816e+07,424.0,1.263909e+08,1.384675e+09,2.368584e+09,2.368584e+09,{'ethereum': '0x0c7d5ae016f806603cb1782bea29ac...
benqi,qi,BENQI,0.009222,3.843076e+07,606.0,6.559338e+07,4.218436e+09,7.200000e+09,7.200000e+09,{'avalanche': '0x8729438eb15e2c8b576fcc6aecda6...
stakewise,swise,StakeWise,0.088043,2.901364e+07,704.0,8.806911e+07,3.294417e+08,1.000000e+09,1.000000e+09,{'ethereum': '0x48c3399719b582dd63eb5aadf12a40...
stader,sd,Stader,0.682945,2.249345e+07,791.0,1.025889e+08,3.288874e+07,1.500000e+08,1.500000e+08,{'ethereum': '0x30d20208d987713f46dfd34ef128bb...


In [6]:
category_data,slippage_data = methodology_class.assess_liquidity()
slippage_data

0x7205705771547cf79201111b4bd8aaf29467b9ec
0x9046d36440290ffde54fe0dd84db8b1cfee9107b
0x9eaf8c1e34f05a589eda6bafdf391cf6ad3cb239
0xaeaeed23478c3a4b798e4ed40d8b7f41366ae861
0xdf474b7109b73b7d57926d43598d5934131136b2
0xaeaeed23478c3a4b798e4ed40d8b7f41366ae861
0x84c882a4d8eb448ce086ea19418ca0f32f106117


,spot price,delivery price,slippage,blockchain
id,,,,
rocket-pool,0.032089,0.032045,-0.001354,ethereum
lido-dao,0.421561,0.417110,-0.010560,ethereum
frax-share,0.116422,0.112946,-0.029852,ethereum
stakewise,11.377897,10.927200,-0.039612,ethereum
stader,1.493245,1.425734,-0.045211,ethereum
yearn-finance,0.000118,0.000111,-0.053879,ethereum
benqi,108.831705,100.033139,-0.080846,avalanche
yield-yak,0.001510,0.001293,-0.144182,avalanche
stake-dao,2.677988,2.104094,-0.214301,ethereum


In [7]:
#methodology_class.check_redstone_price_feeds()

In [8]:
methodology_class.calculate_weights()

,lido-dao,frax-share,rocket-pool,yearn-finance,benqi,stakewise,stader
date,,,,,,,
2023-12-06,0.3,0.279601,0.26216,0.121109,0.015432,0.012208,0.009491


In [9]:
methodology_class.converted_weights()

lido-dao         77
frax-share       72
rocket-pool      67
yearn-finance    31
benqi             3
stakewise         3
stader            2
Name: 2023-12-06 00:00:00, dtype: int64

In [10]:
methodology_class.show_results()

,name,market_cap,weight,weight_converted,address,blockchain_with_highest_liq
id,,,,,,
lido-dao,Lido DAO,2.105527e+09,0.300000,77,0x5a98fcbea516cf06857215779fd812ca3bef1b32,ethereum
frax-share,Frax Share,6.519259e+08,0.279601,72,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,ethereum
rocket-pool,Rocket Pool,6.127544e+08,0.262160,67,0xd33526068d116ce69f19a9ee46f0bd304f21a51f,ethereum
yearn-finance,yearn.finance,2.842358e+08,0.121109,31,0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e,ethereum
benqi,BENQI,3.843076e+07,0.015432,3,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5,avalanche
stakewise,StakeWise,2.901364e+07,0.012208,3,0x48c3399719b582dd63eb5aadf12a40b4c3f52fa2,ethereum
stader,Stader,2.249345e+07,0.009491,2,0x30d20208d987713f46dfd34ef128bb16c404d10f,ethereum
